In [2]:
import os
import os.path
import numpy as np
from numpy import log, exp, pi
import pandas as pd
import scipy
import random
from scipy.stats import gaussian_kde, loguniform, gamma
from math import lgamma
from tqdm import tqdm
from ast import literal_eval
from glob import glob
from tqdm import tqdm
from itertools import zip_longest
import numpy.ma as ma # for masked arrays
from astropy.table import Table, join
import astropy.coordinates as coord
import astropy.units as u
import gala.dynamics as gd
import gala.potential as gp
from pyia import GaiaData

# these packages are for fitting with numpyro
import numpyro
from numpyro import distributions as dist, infer
import numpyro_ext
import arviz as az
import jax

# these are psps imports
from psps.transit_class import Population, Star
import psps.simulate_helpers as simulate_helpers
import psps.simulate_transit as simulate_transit
import psps.utils as utils

# plotting imports
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
matplotlib.rcParams.update({'errorbar.capsize': 1})
pylab_params = {'legend.fontsize': 'large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
pylab.rcParams.update(pylab_params)

import warnings
warnings.filterwarnings("ignore")

path = '/Users/chrislam/Desktop/psps/' 

# we're gonna need this for reading in the initial Berger+ 2020 data
def literal_eval_w_exceptions(x):
    try:
        return literal_eval(str(x))   
    except Exception as e:
        pass

In [8]:
import math
from astropy.coordinates import SkyCoord, Galactic

def convert_ra_dec_to_b(ra, dec):
	# Create a SkyCoord object in the ICRS (equatorial) frame
	# ICRS is the standard J2000 equatorial system assumed by default
	c_icrs = SkyCoord(ra=ra * u.degree, dec=dec * u.degree, frame='icrs')

	# Transform the coordinates to the Galactic frame
	c_galactic = c_icrs.transform_to(Galactic())
	# or use the shorthand attribute access:
	# c_galactic = c_icrs.galactic

	# The Galactic latitude 'b' is the angle from the Galactic midplane (b=0)
	b = c_galactic.b * u.degree
	
	return np.abs(b.value)

# K2 campaign pointings, from https://archive.stsci.edu/missions-and-data/k2/campaign-fields
ras = [173.939610, 246.1264, 336.66534641439, 59.0759116, 130.1576478, 204.8650344, 287.82850661, 16.3379975, 270.3544823, 186.7794430,
	   260.3880064, 351.6588124, 72.7971166, 160.6824762, 233.6175730, 133.7099689, 202.5496152, 
	   130.1610170, 347.2590265]
decs = [1.4172989, -22.4473, -11.096663792177, 18.6605794, 16.8296140, -11.2953585, -23.36001815, 5.2623459, -21.7798098, -4.0271572,
		-23.9759578, -5.1023328, 20.7870759, 6.8509316, -20.0792397, 18.5253931, -7.7210759, 
		16.8278629, -4.2027029]
campaigns = np.arange(19)+1
highs = [1, 3, 6, 8, 10, 12, 14, 17, 19]
lows = [2, 4, 5, 7, 9, 11, 13, 15, 16, 18]
bs = []
for i in range(19):
    bs.append(convert_ra_dec_to_b(ras[i], decs[i]))
baselines = [83, 82, 81, 75, 74, 78, 83, 80, 71, 76, 75, 79, 80, 80, 89, 80, 68, 51, 28]
k2_pointings = dict({'campaign': campaigns, 'ra': ras, 'dec': decs, 'b': bs, 'baseline': baselines})
print(k2_pointings)

def mes(rp, rs, pp, cdpp, baseline):
	"""
	Calculate Multiple Event Statistic, per Eqn 5 of https://iopscience.iop.org/article/10.3847/1538-3881/ac2309#ajac2309t3

	Args:
		rp (float): planet radius, Earth radii
		rs (float): stellar radius, Solar radii
		pp (float): planet period, days
		cdpp (float): CDPP for 6 hr transit duration (or whatever, but keep it the same!)
		baseline (int): K2 campaign length, days

	Returns:
		mes: Multiple Event Statistic, for use in computing detection efficiency
	"""
	
	n_transits = np.floor(baseline/pp)
	depth = (rp/rs)**2
	mes = 0.9488 * (depth/cdpp) * np.sqrt(n_transits)
	return mes

def recovery_fraction(mes, a, k, l):
	"""
	Compute the y value of the logistic recovery efficiency function of Eqn 6 in https://iopscience.iop.org/article/10.3847/1538-3881/ac2309#ajac2309t3

	Args:
		mes (float): Multiple Event Statistic, output of mes()
		a (float): see Table 3 of https://iopscience.iop.org/article/10.3847/1538-3881/ac2309#ajac2309t3
		k (float): see Table 3 of https://iopscience.iop.org/article/10.3847/1538-3881/ac2309#ajac2309t3
		l (float): see Table 3 of https://iopscience.iop.org/article/10.3847/1538-3881/ac2309#ajac2309t3

	Returns:
		f: recovery fraction
	"""

	denominator = 1 + math.exp(-k*(mes-l))
	f = a/denominator
	return f

{'campaign': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]), 'ra': [173.93961, 246.1264, 336.66534641439, 59.0759116, 130.1576478, 204.8650344, 287.82850661, 16.3379975, 270.3544823, 186.779443, 260.3880064, 351.6588124, 72.7971166, 160.6824762, 233.617573, 133.7099689, 202.5496152, 130.161017, 347.2590265], 'dec': [1.4172989, -22.4473, -11.096663792177, 18.6605794, 16.829614, -11.2953585, -23.36001815, 5.2623459, -21.7798098, -4.0271572, -23.9759578, -5.1023328, 20.7870759, 6.8509316, -20.0792397, 18.5253931, -7.7210759, 16.8278629, -4.2027029], 'b': [58.531292937220314, 18.532959446605993, 52.42647045809778, 25.947332351078384, 31.51387163024061, 49.84903553228032, 14.570462204991587, 57.43518020338808, 0.5592168377172165, 58.29587273157315, 7.211487713852137, 60.11396589334712, 14.77267573551696, 53.350902824912026, 28.464777359996905, 35.2829262192038, 53.90825186787161, 31.51620958465847, 56.49896205637861], 'baseline': [83, 82, 81, 75, 

In [3]:
k2_stars_bootstrapped = pd.read_csv(path+'data/k2/k2_stars_bootstrapped.csv')
tri_matched_to_b23_k2 = pd.read_csv(path+'data/k2/tri_matched_to_b23_k2.csv')

In [4]:
print(list(k2_stars_bootstrapped.columns))
print(list(tri_matched_to_b23_k2.columns))

['id_starname', 'iso_teff', 'iso_teff_err1', 'iso_teff_err2', 'iso_logg', 'iso_logg_err1', 'iso_logg_err2', 'iso_feh', 'iso_feh_err1', 'iso_feh_err2', 'iso_mass', 'iso_mass_err1', 'iso_mass_err2', 'iso_rad', 'iso_rad_err1', 'iso_rad_err2', 'iso_rho', 'iso_rho_err1', 'iso_rho_err2', 'iso_lum', 'iso_lum_err1', 'iso_lum_err2', 'iso_age', 'iso_age_err1', 'iso_age_err2', 'iso_dis', 'iso_dis_err1', 'iso_dis_err2', 'iso_avs', 'EPIC', 'dr3_source_id', 'bpmag', 'bpmag_err', 'rpmag', 'rpmag_err', 'parallax_x', 'parallax_err', 'feh', 'feh_err', 'feh_prov', 'ruwe', 'SOURCE_ID', 'ra', 'dec', 'parallax_y', 'pmra', 'pmdec', 'radial_velocity', 'ra_error', 'dec_error', 'parallax_error', 'pmra_error', 'pmdec_error', 'radial_velocity_error', 'parallax', 'zmax', 'age', 'Teff', 'logg']
['index', '#Gc', 'logAge', '[M/H]', 'm_ini', 'logL', 'logTe', 'logg', 'm-M0', 'Av', 'm2/m1', 'mbol', 'Kepler', 'g', 'r', 'i', 'z', 'DDO51_finf', 'J', 'H', 'Ks', 'Mact', 'distance', 'height', 'Teff', 'age', 'stellar_radius', 

In [13]:
# We need CDPP to calculate sensitivity function
# Table 2 of Scaling K2 pt 4 paper has CDPPs! Zink+21: https://iopscience.iop.org/article/10.3847/1538-3881/ac2309#ajac2309t2
k2_cdpps = pd.read_csv(path+'data/k2/scalingk2_iv_cdpps.txt',sep='\s+')
print(k2_cdpps)
k2_stars_cdpps = pd.merge(k2_stars_bootstrapped, k2_cdpps[['EPIC', 'CDPP60']], on='EPIC')
print(k2_stars_cdpps)

             EPIC  CAMP    CDPP10    CDPP15    CDPP20    CDPP25    CDPP30  \
0       201121245     1   257.151   236.598   226.563   209.841   191.452   
1       201122454     1   268.578   248.491   225.499   214.708   198.934   
2       201122521     1   486.386   450.572   368.257   355.254   328.753   
3       201123619     1   639.118   587.075   533.719   479.562   471.031   
4       201124136     1   102.980    96.421    87.393    88.527    81.895   
...           ...   ...       ...       ...       ...       ...       ...   
372177  251813733    18   152.999   124.462   114.270   105.747    95.165   
372178  251813735    18   169.818   148.135   147.267   137.272   135.030   
372179  251813736    18  1716.881  1385.709  1229.793  1160.628  1009.841   
372180  251813737    18   606.881   482.671   460.517   389.975   422.369   
372181  251813738    18    69.956    57.544    52.358    49.004    42.763   

         CDPP40   CDPP50   CDPP60   CDPP70   CDPP80   CDPP90  CDPP100  
0  

In [3]:
# planet formation history model parameters
threshold = 12 # cosmic age in Gyr; 13.7 minus stellar age, then round
frac1 = 0.20 # frac1 must be < frac2 if comparing cosmic ages
frac2 = 0.95

name_thresh = 12
name_f1 = 20
name_f2 = 95
name = 'step_'+str(name_thresh)+'_'+str(name_f1)+'_'+str(name_f2)

pop = Population(tri_matched_to_b23_k2['age'], threshold, frac1, frac2)
frac_hosts = pop.galactic_occurrence_step(threshold, frac1, frac2)
#frac_hosts = pop.galactic_occurrence_monotonic(frac1, frac2)
#frac_hosts = pop.galactic_occurrence_piecewise(frac1, frac2, threshold)
intact_fracs = scipy.stats.truncnorm.rvs(0, 1, loc=0.18, scale=0.1, size=len(tri_matched_to_b23_k2))  # np vs JAX bc of random key issues

alpha_se = np.random.normal(-1., 0.2)
alpha_sn = np.random.normal(-1.5, 0.1)

# create Star objects, with their planetary systems
star_data = []
for i in tqdm(range(len(tri_matched_to_b23_k2))): 
    star = Star(tri_matched_to_b23_k2['age'][i], tri_matched_to_b23_k2['stellar_radius'][i], tri_matched_to_b23_k2['Mact'][i], tri_matched_to_b23_k2['Teff'], tri_matched_to_b23_k2['cdpp'][i], tri_matched_to_b23_k2['height'][i], alpha_se, alpha_sn, frac_hosts[i], intact_fracs[i])
    star_update = {
        'kepid': star.kepid,
        'age': star.age,
        'stellar_radius': star.stellar_radius,
        'stellar_mass': star.stellar_mass,
        'Teff': star.Teff,
        'rrmscdpp06p0': star.rrmscdpp06p0,
        'frac_host': star.frac_host,
        'height': star.height,
        'midplane': star.midplane,
        'prob_intact': star.prob_intact,
        'status': star.status,
        'sigma_incl': star.sigma_incl,
        'num_planets': star.num_planets,
        'periods': star.periods,
        'incls': star.incls,
        'mutual_incls': star.mutual_incls,
        'eccs': star.eccs,
        'omegas': star.omegas,
        'planet_radii': star.planet_radii
    }
    star_data.append(star_update)
    pop.add_child(star)

# convert back to DataFrame
#j = 0
trilegal_k2 = pd.DataFrame.from_records(star_data)

"""
# why are physical occurrences rising at greater heights all of a sudden??
trilegal_kepler_all['height_bins'] = pd.cut(trilegal_kepler_all['height'], bins = np.logspace(2,3,6), include_lowest=True)
trilegal_physical_counts = trilegal_kepler_all.groupby(['height_bins'])['kepid'].count()
print(trilegal_physical_counts)
print(trilegal_physical_counts['kepid'])
quit()
trilegal_physical_counts = trilegal_physical_counts[['height_bins',0]]
trilegal_physical_counts.columns = ['height_bins', 'height_bin_counts']
print(np.array(trilegal_physical_counts['height_bin_counts']))
quit()
"""

#trilegal_k2.to_csv(path+'data/trilegal_k2/'+name+'/'+name+'_'+str(j)+'.csv')

  0%|          | 0/95160 [00:00<?, ?it/s]


KeyError: 'cdpp'